In [103]:
from typing import cast, List

import copy
import rich
import torch
import stanza
import pandas as pd
from stanza import Document
from rich.console import Console
from stanza.models.common.doc import Token, Word

# stanza.download('pt') # download Portuguese model

torch.cuda.is_available()

True

In [2]:
console = Console()

# GramatiQuizz

> Esta aplicação busca coletar dados para uma pesquisa de mestrado na Universidade de São Paulo.

Bem vindo ao GramatiQuizz. Instruções:

1. Selecionar sua série;
2. Ler a frase proposta para o teste;
3. Para cada palavra da frase você deve selecionar a classe morfossintática que ela pertence;

Considerações:
- Não se preocupe se você errar;
- Você pode responder a ordem que quiser.



# Universal Dependencies TAGs to Portuguese

| UPOS | Portuguese | English |
|------|------------|---------|
| ADJ | Adjetivo | Adjective |
| ADP | Preposição | Adposition |
| ADV | Advérbio | Adverb |
| AUX | Verbo Auxiliar ou de Ligação $^1$ | Auxiliary |
| CCONJ | Conjunção Coordenativa | Coordinating Conjunction |
| DET | Artigo | Determiner |
| INTJ | Interjeição | Interjection |
| NOUN | Substantivo | Noun |
| NUM | Numeral | Numeral |
| PART | Partícula | Particle |
| PRON | Pronome | Pronoun |
| PROPN | Nome Próprio | Proper Noun |
| PUNCT | Pontuação | Punctuation |
| SCONJ | Conjunção Subordinativa | Subordinating Conjunction |
| SYM | Símbolo | Symbol |
| VERB | Verbo | Verb |
| X | Outro | Other |


$^1$ If `deprel` = `AUX`, it is "Verbo Auxiliar" | If `deprel` = `cop` (from `copula`), it is "Verbo de Ligação" 

# Samples
- `ADV`     : $\text{Ele normalmente come café da manhã antes de ir trabalhar.}$
- `ADV`     : "Ela frequentemente se atrasava para a reunião, o que gerava alguma confusão." 
- `AUX`     : "Luís havia passado a noite em claro." ("Verbo Auxiliar")
- `AUX`     : "Ele parece estar cansado." ("Verbo de Ligação")
- `CCONJ`   : "Antonio estudou muito, portanto tirou uma boa nota."
- `INTJ`    : "Oi, você pode me ajudar?"
- 


# Stanza 101

## Document Methods

| Method | Description |
|--------|-------------|
| `add_property` | Add a property to the document |
| `build_ents` | Build entities in the document |
| `coref` | Access coreference information |
| `entities` | Get entities from the document |
| `ents` | Access named entities |
| `from_serialized` | Create document from serialized data |
| `get` | Get a property value |
| `get_mwt_expansions` | Get multi-word token expansions |
| `iter_tokens` | Iterate over tokens in the document |
| `iter_words` | Iterate over words in the document |
| `lang` | Get document language |
| `mark_whitespace` | Mark whitespace in the document |
| `num_tokens` | Get total number of tokens |
| `num_words` | Get total number of words |
| `reindex_sentences` | Reindex sentences in the document |
| `sentence_comments` | Access sentence comments |
| `sentences` | Access sentences in the document |
| `set` | Set a property value |
| `set_mwt_expansions` | Set multi-word token expansions |
| `sort_features` | Sort morphological features |
| `text` | Get raw text of the document |
| `to_dict` | Convert document to dictionary |
| `to_serialized` | Convert document to serialized format |

## Token Methods

| Method | Description |
|--------|-------------|
| `add_property` | Add a property to the token |
| `consolidate_whitespace` | Consolidate whitespace in the token |
| `end_char` | Get the ending character position |
| `id` | Get the token ID |
| `is_mwt` | Check if token is a multi-word token |
| `manual_expansion` | Access manual expansion data |
| `misc` | Access miscellaneous token information |
| `multi_ner` | Access multi-layer named entity recognition |
| `ner` | Access named entity recognition tag |
| `pretty_print` | Print token in a formatted way |
| `sent` | Get the sentence containing this token |
| `spaces_after` | Get spaces after the token |
| `spaces_before` | Get spaces before the token |
| `start_char` | Get the starting character position |
| `text` | Get the token text |
| `to_conll_text` | Convert token to CoNLL format |
| `to_dict` | Convert token to dictionary |
| `words` | Access words within the token |

In [3]:
nlp = stanza.Pipeline('pt') # initialize English neural pipeline

2025-05-25 16:53:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-05-25 16:53:08 INFO: Downloaded file to /home/antonio/stanza_resources/resources.json
2025-05-25 16:53:11 INFO: Loading these models for language: pt (Portuguese):
| Processor    | Package         |
----------------------------------
| tokenize     | bosque          |
| mwt          | bosque          |
| pos          | bosque_charlm   |
| lemma        | bosque_nocharlm |
| constituency | cintil_charlm   |
| depparse     | bosque_charlm   |

2025-05-25 16:53:11 INFO: Using device: cuda
2025-05-25 16:53:11 INFO: Loading: tokenize
2025-05-25 16:53:13 INFO: Loading: mwt
2025-05-25 16:53:13 INFO: Loading: pos
2025-05-25 16:53:15 INFO: Loading: lemma
2025-05-25 16:53:15 INFO: Loading: constituency
2025-05-25 16:53:15 INFO: Loading: depparse
2025-05-25 16:53:16 INFO: Done loadi

In [58]:

phrase = "Ele normalmente come café da manhã antes de ir trabalhar."
phrase = "Ele parece estar cansado."
phrase = "Luís havia passado a noite em claro."
phrase = "Antonio estudou muito, portanto tirou uma boa nota."
phrase = "Os meus livros estão na estante."
phrase = "Oi, você pode me ajudar?"
phrase = "Embora ficasse nervosa, sempre se saía bem."
phrase = "Reunião adiada por causa da chuva."

phrase = str(input("Digite uma frase: "))

d:Document = nlp(phrase)

print(f"Num tokens: {d.num_tokens} | Num words: {d.num_words}")

token:Token
for token in d.iter_tokens():
    words:List[Word] = token.words
    console.print(token)
    print(f'----------------{type(token)}-------------------')


Num tokens: 5 | Num words: 5


[
  {
    "id": 1,
    "text": "Eles",
    "lemma": "eles",
    "upos": "PRON",
    "feats": "Case=Nom|Gender=Masc|Number=Plur|Person=3|PronType=Prs",
    "head": 2,
    "deprel": "nsubj",
    "start_char": 0,
    "end_char": 4
  }
]

----------------<class 'stanza.models.common.doc.Token'>-------------------


[
  {
    "id": 2,
    "text": "têm",
    "lemma": "ter",
    "upos": "VERB",
    "feats": "Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "start_char": 5,
    "end_char": 8
  }
]

----------------<class 'stanza.models.common.doc.Token'>-------------------


...

----------------<class 'stanza.models.common.doc.Token'>-------------------


[
  {
    "id": 4,
    "text": "poder",
    "lemma": "poder",
    "upos": "NOUN",
    "feats": "Gender=Masc|Number=Sing",
    "head": 2,
    "deprel": "obj",
    "start_char": 11,
    "end_char": 16,
    "misc": "SpaceAfter=No"
  }
]

----------------<class 'stanza.models.common.doc.Token'>-------------------


[
  {
    "id": 5,
    "text": ".",
    "lemma": ".",
    "upos": "PUNCT",
    "head": 2,
    "deprel": "punct",
    "start_char": 16,
    "end_char": 17,
    "misc": "SpaceAfter=No"
  }
]

----------------<class 'stanza.models.common.doc.Token'>-------------------


In [43]:
[m for m in dir(token) if not m.startswith('_')]

['add_property',
 'consolidate_whitespace',
 'end_char',
 'id',
 'is_mwt',
 'manual_expansion',
 'misc',
 'multi_ner',
 'ner',
 'pretty_print',
 'sent',
 'spaces_after',
 'spaces_before',
 'start_char',
 'text',
 'to_conll_text',
 'to_dict',
 'words']

In [26]:
[m for m in dir(d) if not m.startswith('_')]

['add_property',
 'build_ents',
 'coref',
 'entities',
 'ents',
 'from_serialized',
 'get',
 'get_mwt_expansions',
 'iter_tokens',
 'iter_words',
 'lang',
 'mark_whitespace',
 'num_tokens',
 'num_words',
 'reindex_sentences',
 'sentence_comments',
 'sentences',
 'set',
 'set_mwt_expansions',
 'sort_features',
 'text',
 'to_dict',
 'to_serialized']

In [63]:
upos_translation = {
    "ADJ"  : "Adjetivo",
    "ADP"  : "Preposição",
    "ADV"  : "Advérbio",
    "AUX"  : {'pass':"Verbo Passivo", 'cop':"Verbo de Ligação", 'aux:pass':"Verbo Auxiliar", 'aux':"Verbo Auxiliar"},
    "CCONJ": "Conjunção Coordenativa",
    "DET"  : "Artigo",
    "INTJ" : "Interjeição",
    "NOUN" : "Substantivo",
    "NUM"  : "Numeral",
    "PART" : "Partícula",
    "PRON" : "Pronome",
    "PROPN": "Nome Próprio",
    "PUNCT": "Pontuação",
    "SCONJ": "Conjunção Subordinativa",
    "SYM"  : "Símbolo",
    "VERB" : "Verbo",
    "X"    : "Outro"
}

dict_values(['Adjetivo', 'Preposição', 'Advérbio', {'pass': 'Verbo Passivo', 'cop': 'Verbo de Ligação', 'aux:pass': 'Verbo Auxiliar', 'aux': 'Verbo Auxiliar'}, 'Conjunção Coordenativa', 'Artigo', 'Interjeição', 'Substantivo', 'Numeral', 'Partícula', 'Pronome', 'Nome Próprio', 'Pontuação', 'Conjunção Subordinativa', 'Símbolo', 'Verbo', 'Outro'])

In [95]:
def get_translated_tag(word: Word) -> str:
    """
    Retorna a tradução da tag UPOS para português.
    """
    if not str(word.upos) == 'AUX':
        return upos_translation[str(word.upos)]
    else:
        try:
            return upos_translation['AUX'][str(word.deprel)]
        except KeyError as e:
            print(f"KeyError: {e}")
            return upos_translation['AUX']['aux']

def get_pos_tags(text) -> list[dict[str, str]]:
    """
    Processa o texto e retorna uma lista de dicionários com a palavra e sua classe gramatical em português.
    """
    doc:Document = nlp(text)
    pos_tags = []

    token:Token
    for token in doc.iter_tokens():
        words:List[Word] = token.words
        if not token.is_mwt():
            word = words[0]
            translated_tag = get_translated_tag(word)  # Se não achar, usa a própria tag original
            w = {
                'word': token.text,
                'tag': translated_tag
            }
        else:
            w = {
                'word': token.text,
                'tag': '+'.join([get_translated_tag(w) for w in words])
            }

        if w['tag'] == 'Outro':
            return []
        pos_tags.append(w)

    return pos_tags

In [82]:
# phrase = "Com certeza você não está dizendo a verdade, não é mesmo?"
# nlp(phrase)

In [97]:
import json

phrase = "E então, sem que ninguém esperasse, ele simplesmente disse: 'enough!'."

tags_list = get_pos_tags(phrase)

if not tags_list:
    print("Nenhum token encontrado.")
else:
    d = {
        'phrase': phrase,
        'pos_tags': tags_list
    }

    # Printing the dictionary as formatted JSON
    print(json.dumps(d, indent=2, ensure_ascii=False))

Nenhum token encontrado.


In [29]:
from sympy import Li

phrase = "Reunião adiada por causa da chuva."

d:Document = nlp(phrase)

t:Token
for t in d.iter_tokens():
    words:List[Word] = t.words
    assert isinstance(words, list)
    if t.is_mwt(): # is multi-word token
        w = {
            'word': t.text,
            'tag': '+'.join([get_translated_tag(w) for w in words])
        }
        print(w)

{'word': 'da', 'tag': 'Preposição+Artigo'}


# Read pickle

In [ ]:
import pickle

with open('data/frase_tags.pkl', 'rb') as f:
    frase_tags = pickle.load(f)

In [80]:
frase_tags[frase_tags['frase'].str.contains('enough')]['frase'].values[0]

"E então, sem que ninguém esperasse, ele simplesmente disse: 'enough!'."

In [100]:
frase_tags.drop_duplicates(subset=['frase'], inplace=True)

In [101]:
frase_tags = cast(pd.DataFrame, frase_tags)

d = []

for i,row in frase_tags.iterrows():
    word_pos_list = []
    for word in row['tags'].iter_words():
        translated_pos = upos_translation.get(word.upos, word.upos)
        word_pos_list.append({"word": word.text, "tag": translated_pos})

    try:
        tags_list = get_pos_tags(row['frase'])
        if not tags_list:
            print(f"Phrase skipped: {row['frase']}")
            continue
        s = {
            'pharase': row['frase'],
            'tags' : tags_list,
        }
    except Exception as e:
        print(f"Error processing phrase: {row['frase']}")
        raise e

    d.append(s)

Phrase skipped: E então, sem que ninguém esperasse, ele simplesmente disse: 'enough!'.
KeyError: 'conj'


In [104]:
addon_phrases = [
    "Ele normalmente come café da manhã antes de ir trabalhar.",
    "Ele parece estar cansado.",
    "Luís havia passado a noite em claro.",
    "Antonio estudou muito, portanto tirou uma boa nota.",
    "Os meus livros estão na estante.",
    "Oi, você pode me ajudar?",
    "Embora ficasse nervosa, sempre se saía bem.",
    "Reunião adiada por causa da chuva."
]

def add_more_phrases(d:List[dict[str, str]], phrases:List[str]) -> List[dict[str, str]]:
    d_copy = copy.deepcopy(d)

    for phrase in addon_phrases:
        try:
            tags_list = get_pos_tags(phrase)
            if not tags_list:
                print(f"Phrase skipped: {phrase}")
                continue
            s = {
                'pharase': phrase,
                'tags' : tags_list,
            }
        except Exception as e:
            print(f"Error processing phrase: {phrase}")
            raise e
        
        d_copy.append(s)

    return d_copy
    
d_add = add_more_phrases(d, addon_phrases)

In [105]:
import json

with open('data/processed_phrases_2.json', 'w', encoding='utf-8') as f:
    json.dump(d, f, indent=2, ensure_ascii=False)


In [35]:
word_pos_list = []
for word in x.iter_words():
    translated_pos = upos_translation.get(word.upos, word.upos)
    word_pos_list.append({"word": word.text, "tag": translated_pos})

word_pos_list

[{'word': 'O', 'tag': 'Determinante'},
 {'word': 'gato', 'tag': 'Substantivo'},
 {'word': 'dorme', 'tag': 'Verbo'},
 {'word': 'em', 'tag': 'Preposição'},
 {'word': 'o', 'tag': 'Determinante'},
 {'word': 'sofá', 'tag': 'Substantivo'},
 {'word': '.', 'tag': 'Pontuação'}]